In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
!ls dataset/

categories_atp.csv  links.csv	     ratings_atp.csv  today_atp.csv
categories_mxt.csv  model	     ratings_wta.csv  today_mxt.csv
categories_wta.csv  odds_atp.csv     README.md	      today_wta.csv
courts.csv	    odds_mxt.csv     rounds.csv       tours_atp.csv
facts_atp.csv	    odds_wta.csv     seed_atp.csv     tours_mxt.csv
facts_wta.csv	    pairs.csv	     seed_mxt.csv     tours_wta.csv
games_atp.csv	    players_atp.csv  seed_wta.csv     url.csv
games_mxt.csv	    players.csv      sqlite.ipynb     version.csv
games_wta.csv	    players_mxt.csv  stat_atp.csv
injury_atp.csv	    players_wta.csv  stat_mxt.csv
injury_wta.csv	    points.csv	     stat_wta.csv


In [3]:
# захерачить это в самом верху, чтобы cursor был глобальной переменной, и делать con.close() только в самом конце

import sqlite3
con = sqlite3.connect('on_court.db')
cursor = con.cursor()

cursor.execute("SELECT * FROM sqlite_master")
schema = cursor.fetchall() # можно вывести схему (правда там нет forign key-ев)

query = 'select * from games_atp where ID1_G={id_}'
query = query.format(id_=115)
cursor.execute(query).fetchone()
# >> (115, 105, 1, 12, '6-2 6-1', None)

# если делали alter/create/insert into table:
# con.commit()



(115, 105, 1, 12, u'6-2 6-1', None)

In [4]:
query = 'select * from games_atp where ID1_G={id}'
query = query.format(id=115)
cursor.execute(query).fetchone()

(115, 105, 1, 12, u'6-2 6-1', None)

In [5]:
cursor.execute("SELECT * FROM sqlite_master")
schema = cursor.fetchall()

In [6]:
for i in range(len(schema)):
    print()
    table_name = schema[i][1]
    table_columns = schema[i][-1]
    cols = table_columns.split('(')[1][:-1]
    cols = list(map(lambda r: r.strip().split('			')[0].replace('`', ''), cols.split(',')))
    print(table_name)
    print(' | '.join(cols))
    for col in cols:
        if 'ID' in col:
            statement = "CREATE INDEX IF NOT EXISTS index_{1}_{0} ON {0}({1});".format(table_name, col)
            try:
                cursor.execute(statement)
            except Exception as e:
                print('! error in index statement', e)
            print(statement)
            
    

()
categories_atp
ID_P | CAT1 | CAT2 | CAT3 | CAT4 | CAT5 | CAT6 | CAT7 | CAT8 | CAT9 | CAT10 | CAT11 | CAT12 | CAT13 | CAT14 | CAT15 | CAT16 | CAT17 | CAT18 | CAT19 | CAT20 | CAT21 | CAT22 | CAT23 | CAT24 | CAT25 | CAT26 | CAT27 | CAT28 | CAT29 | CAT30
CREATE INDEX IF NOT EXISTS index_ID_P_categories_atp ON categories_atp(ID_P);
()
categories_mxt
ID_P | CAT1 | CAT2 | CAT3 | CAT4 | CAT5 | CAT6 | CAT7 | CAT8 | CAT9 | CAT10 | CAT11 | CAT12 | CAT13 | CAT14 | CAT15 | CAT16 | CAT17 | CAT18 | CAT19 | CAT20 | CAT21 | CAT22 | CAT23 | CAT24 | CAT25 | CAT26 | CAT27 | CAT28 | CAT29 | CAT30
CREATE INDEX IF NOT EXISTS index_ID_P_categories_mxt ON categories_mxt(ID_P);
()
categories_wta
ID_P | CAT1 | CAT2 | CAT3 | CAT4 | CAT5 | CAT6 | CAT7 | CAT8 | CAT9 | CAT10 | CAT11 | CAT12 | CAT13 | CAT14 | CAT15 | CAT16 | CAT17 | CAT18 | CAT19 | CAT20 | CAT21 | CAT22 | CAT23 | CAT24 | CAT25 | CAT26 | CAT27 | CAT28 | CAT29 | CAT30
CREATE INDEX IF NOT EXISTS index_ID_P_categories_wta ON categories_wta(ID_P);
()
f

In [7]:
con.commit()

In [8]:
con.close()

In [30]:
con = sqlite3.connect("on_court.db")
cur = con.cursor()
cur.execute("DROP TABLE players")
con.commit()

In [31]:
import csv, sqlite3

con = sqlite3.connect("on_court.db")
cur = con.cursor()
cur.execute("CREATE TABLE players (id, rating, rank, results1, results2, results3, results4, results5, games, winrate, after_game_period);") # use your column names here

fin = open('players.csv','rb') # `with` statement available in 2.5+
# csv.DictReader uses first line in file for column headings by default
dr = csv.DictReader(fin) # comma is default delimiter

to_db = [(i['id'], i['rating'], i['rank'], i['results1'], i['results2'], i['results3'], i['results4'], i['results5'], i['games'], i['winrate'], i['after_game_period']) for i in dr]

for st in to_db:
    cur.execute("INSERT INTO players (id, rating, rank, results1, results2, results3, results4, results5, games, winrate, after_game_period) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);", st)
con.commit()
con.close()



In [32]:
con = sqlite3.connect('on_court.db')
cursor = con.cursor()


query = 'select * from players'
cursor.execute(query).fetchone()
con.commit()

In [33]:
con.close()

In [36]:
con = sqlite3.connect("on_court.db")
cur = con.cursor()
cur.execute("DROP TABLE pairs")
con.commit()

In [37]:
con = sqlite3.connect("on_court.db")
cur = con.cursor()
cur.execute("CREATE TABLE pairs (id_p, rating_diff,rank_diff,games_diff,winrate_diff,after_game_period_diff,meetings_count,pair_winrate,meeting_results1,meeting_results2,meeting_results3,meeting_results4,meeting_results5,indirect_score);") # use your column names here

fin = open('pairs.csv','rb') # `with` statement available in 2.5+
# csv.DictReader uses first line in file for column headings by default
dr = csv.DictReader(fin) # comma is default delimiter

to_db = [(i['id_p'], i['rating_diff'],i['rank_diff'],i['games_diff'],i['winrate_diff'],i['after_game_period_diff'],i['meetings_count'],i['pair_winrate'],i['meeting_results1'],i['meeting_results2'],i['meeting_results3'],i['meeting_results4'],i['meeting_results5'],i['indirect_score']) for i in dr]

cur.executemany("INSERT INTO pairs (id_p, rating_diff,rank_diff,games_diff,winrate_diff,after_game_period_diff,meetings_count,pair_winrate,meeting_results1,meeting_results2,meeting_results3,meeting_results4,meeting_results5,indirect_score) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);", to_db)
con.commit()
con.close()

In [38]:
con = sqlite3.connect('on_court.db')
cursor = con.cursor()

query = 'select * from pairs'
cursor.execute(query).fetchone()

(u'(85, 89)',
 u'-3531.0',
 u'29.0',
 u'-115.0',
 u'-0.17470266795242684',
 u'0.0',
 u'3.0',
 u'0.6666666666666667',
 u'',
 u'',
 u'1.0',
 u'0.0',
 u'1.0',
 u'1.2592592592592593')

In [39]:
con.commit()
con.close()